In [7]:
import pandas as pd
from tqdm import tqdm



In [3]:
xl = pd.ExcelFile('../datasets/OneDrive_1_5-10-2021/2. XN_Sinh hóa máu/SINH HOA-VHH2013-2019.xlsx')


In [4]:
xl.sheet_names

['SH-2013', 'SH-2015', 'SH-2016', 'Sh-2017', 'Sh-2018', 'Sh-2019', 'Sh-2020']

In [7]:
for e in tqdm(xl.sheet_names):
    df = xl.parse(e)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [03:25<00:00, 29.29s/it]


In [27]:
from os import name
import pandas as pd
from tqdm import tqdm

path = "../datasets/pid.csv"
df = pd.read_csv(path)
flag_dict = {e.strip(): True for e in df['PID']}
columns = ['intime', 'PID', 'SID', 'Age', 'sex', 'Address', 'LocationID', 'LocationName', 'Testname', 'Testcode', 'Result']
pattern = None

def handle_group(ex_df):
    global pattern
    obj = pattern.copy()
    df2 = ex_df[['intime', 'Testcode', 'Result']]
    df2 = df2.assign(indate=df2['intime'].str.split().str[0])
    df2.apply(pd.to_numeric, errors='coerce', downcast='float')
#     df2.dropna(inplace=True)
    m = 0
    selected_df = None
    for date, sub_df in df2.groupby('indate'):
        if m <= sub_df.shape[0]:
            m = sub_df.shape[0]
            obj['indate'] = date
            selected_df = sub_df

    main_df = selected_df[['Testcode','Result']]
    main_dict = main_df.set_index('Testcode').to_dict()['Result']
    obj.update(main_dict)
    return obj

def process_dataframe(df, des_path):
    out_arr = []
    for pid, sub_df in tqdm(df.groupby("PID")):
        tgt = handle_group(sub_df)
        tgt['PID'] = pid
        out_arr.append(tgt)
    
    tgt_df = pd.DataFrame(out_arr).set_index("PID")
    tgt_df.to_csv(des_path)

In [29]:
# df['intime'].str.split().str[0]

df = pd.read_csv("../datasets/OneDrive_1_5-10-2021/2. XN_Sinh hóa máu/SH-2013.csv")
pattern = {e: None for e in df.Testcode.unique()}
process_dataframe(df, 'out.csv')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16465/16465 [00:59<00:00, 275.08it/s]
